In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score

from transformers import ElectraConfig, ElectraForSequenceClassification, ElectraTokenizer

 # custom config of electra with fewer layers, heads, and a smaller hidden size
custom_config = ElectraConfig(
    hidden_size=64, 
    num_attention_heads=2, 
    intermediate_size=256, 
    num_hidden_layers=6,
    max_position_embeddings=64, 
    vocab_size=30522,
    num_labels=3 
)

small_electra_model = ElectraForSequenceClassification(custom_config)

# make a tokenizer from a pre-trained electra model (same vocabulary)
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

print(small_electra_model)

checkpoint_folder = "./New Folder With Items"

bias_model = small_electra_model
bias_tokenizer = AutoTokenizer.from_pretrained(checkpoint_folder)

# load snli
snli_dataset = load_dataset("snli")
full_train_dataset = snli_dataset["train"]

# tokenize training set
def bias_tokenize_function(example):
    return bias_tokenizer(
        example["premise"], 
        example["hypothesis"], 
        truncation=True, 
        padding="max_length", 
        max_length=57
    )

tokenized_full_train = full_train_dataset.map(bias_tokenize_function, batched=True)

# define training args
bias_training_args = TrainingArguments(
    output_dir="./bias_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16, 
)

# handle non-contiguous tensors
class ContiguousTrainer(Trainer):
    def save_model(self, output_dir=None, **kwargs): 
        for name, param in self.model.named_parameters():
            if not param.is_contiguous():
                param.data = param.data.contiguous()
        super().save_model(output_dir, **kwargs)

# trainer for bias model with contiguous saving
bias_trainer = ContiguousTrainer(
    model=bias_model,
    args=bias_training_args,
    train_dataset=tokenized_full_train,
    eval_dataset=tokenized_full_train
)

bias_trainer.train()

# get predictions on train
bias_predictions = bias_trainer.predict(tokenized_full_train).predictions
predicted_labels = np.argmax(bias_predictions, axis=1)
residuals = full_train_dataset["label"] - predicted_labels

checkpoint_folder = "./New Folder With Items"
main_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_folder)
main_tokenizer = AutoTokenizer.from_pretrained(checkpoint_folder)

def main_tokenize_function(example, index):
    tokens = main_tokenizer(
        example["premise"],
        example["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=57
    )
    tokens["labels"] = residuals[index]
    return tokens

# apply tokenization and residual labels for residuals dataset
tokenized_residuals = full_train_dataset.map(main_tokenize_function, with_indices=True)

tokenized_residuals.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

main_training_args = TrainingArguments(
    output_dir="./main_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16, 
)

test_dataset = snli_dataset["test"]

def test_tokenize_function(example):
    return main_tokenizer(
        example["premise"],
        example["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=57
    )

# apply tokenization to test set
tokenized_test_dataset = test_dataset.map(test_tokenize_function, batched=True)

tokenized_test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# define a function to compute accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

# update the main_trainer with compute_metrics
main_trainer = Trainer(
    model=main_model,
    args=main_training_args,
    train_dataset=tokenized_residuals,
    eval_dataset=tokenized_residuals,
    compute_metrics=compute_metrics 
)

# now evaluate the model on the tokenized test dataset
test_results = main_trainer.evaluate(eval_dataset=tokenized_test_dataset)

# print the entire test_results dictionary to check available keys
print("Evaluation results:", test_results)

# access accuracy if available
if "eval_accuracy" in test_results:
    print(f"Final accuracy on the test set: {test_results['eval_accuracy']:.4f}")
elif "accuracy" in test_results:
    print(f"Final accuracy on test set: {test_results['accuracy']:.4f}")
else:
    print("Error.")

/Users/moni/NLP/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(64, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=64, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-5): 6 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=64, out_features=64, bias=True)
              (key): Linear(in_features=64, out_features=64, bias=True)
              (value): Linear(in_features=64, out_features=64, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_fe

Map:   0%|          | 0/550152 [00:00<?, ? examples/s]

/Users/moni/NLP/venv/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/103155 [00:00<?, ?it/s]

...

{'eval_loss': 0.2353406399488449, 'eval_runtime': 14060.4365, 'eval_samples_per_second': 39.128, 'eval_steps_per_second': 4.891, 'epoch': 2.0}
{'loss': 0.2776, 'grad_norm': 13.653707504272461, 'learning_rate': 1.6555183946488294e-05, 'epoch': 2.01}
{'loss': 0.2614, 'grad_norm': 5.901821136474609, 'learning_rate': 1.6312830206970095e-05, 'epoch': 2.02}
{'loss': 0.2515, 'grad_norm': 7.043242931365967, 'learning_rate': 1.6070476467451895e-05, 'epoch': 2.04}
{'loss': 0.2627, 'grad_norm': 9.907173156738281, 'learning_rate': 1.5828122727933693e-05, 'epoch': 2.05}
{'loss': 0.2696, 'grad_norm': 9.449553489685059, 'learning_rate': 1.558576898841549e-05, 'epoch': 2.06}
{'loss': 0.2645, 'grad_norm': 10.251653671264648, 'learning_rate': 1.534341524889729e-05, 'epoch': 2.08}
{'loss': 0.2587, 'grad_norm': 6.704514980316162, 'learning_rate': 1.510106150937909e-05, 'epoch': 2.09}
{'loss': 0.2673, 'grad_norm': 6.611218452453613, 'learning_rate': 1.4858707769860888e-05, 'epoch': 2.11}
{'loss': 0.2657, '

  0%|          | 0/68769 [00:00<?, ?it/s]

{'eval_loss': 0.20425069332122803, 'eval_runtime': 1442.6806, 'eval_samples_per_second': 381.34, 'eval_steps_per_second': 47.668, 'epoch': 3.0}
{'train_runtime': 66631.681, 'train_samples_per_second': 24.77, 'train_steps_per_second': 1.548, 'train_loss': 0.30845974834398315, 'epoch': 3.0}


  0%|          | 0/1250 [00:00<?, ?it/s]

Evaluation Results: {'eval_loss': 3.4126429557800293, 'eval_model_preparation_time': 0.0023, 'eval_accuracy': 0.4032, 'eval_runtime': 31.2216, 'eval_samples_per_second': 320.291, 'eval_steps_per_second': 40.036}
Final Accuracy on the Test Set: 0.4032
